## Classical to Quantum Circuit
Group S

Fredrik Etsare, fret6070

Jonas Hofmann, joho2035

In [1]:
from qiskit import QuantumCircuit
from ClassicalCircuitExercise import ClassicalCircuit

## 2.3 Step 1

In [2]:
def convert_step_1(self, quantumCircuit):
    for gate in self.gates:
        match gate[1]:
            case "and":
                a, b, c = gate[0], gate[2], gate[3]
                quantumCircuit.ccx(b, c, a)
            case "not":
                a, b = gate[0], gate[2]
                quantumCircuit.x(a)
                quantumCircuit.cx(b, a)
            case "or":
                # bonus, a = or(b,c) = not(and(not b, not c))
                a, b, c = gate[0], gate[2], gate[3]
                quantumCircuit.x(b)
                quantumCircuit.x(c)
                quantumCircuit.ccx(b, c, a)
                quantumCircuit.x(a)
                quantumCircuit.x(b)
                quantumCircuit.x(c)
            case "xor":
                # bonus, cnot kinda is xor
                a, b, c = gate[0], gate[2], gate[3]
                quantumCircuit.cx(b, a)
                quantumCircuit.cx(c, a)
            case "nand":
                # bonus, toffoli is and so not + toffoli = nand
                a, b, c = gate[0], gate[2], gate[3]
                quantumCircuit.ccx(b, c, a)
                quantumCircuit.x(a)
        quantumCircuit.barrier()

ClassicalCircuit.convert_step_1 = convert_step_1

## 2.4 Step 2

In [3]:
def convert_step_2(self, quantumCircuit):
    for gate in reversed(self.gates):
        match gate[1]:
            case "and":
                a, b, c = gate[0], gate[2], gate[3]
                quantumCircuit.ccx(b, c, a)
            case "not":
                a, b = gate[0], gate[2]
                quantumCircuit.cx(b, a)
                quantumCircuit.x(a)
            case "or":
                # bonus, reversed
                a, b, c = gate[0], gate[2], gate[3]
                quantumCircuit.x(c)
                quantumCircuit.x(b)
                quantumCircuit.x(a)
                quantumCircuit.ccx(b, c, a)
                quantumCircuit.x(c)
                quantumCircuit.x(b)
            case "xor":
                # bonus, reversed
                a, b, c = gate[0], gate[2], gate[3]
                quantumCircuit.cx(c, a)
                quantumCircuit.cx(b, a)
            case "nand":
                # bonus, reversed
                a, b, c = gate[0], gate[2], gate[3]
                quantumCircuit.x(a)
                quantumCircuit.ccx(b, c, a)
        quantumCircuit.barrier()


ClassicalCircuit.convert_step_2 = convert_step_2

## 2.5 Step 3

In [4]:
def convert(self, quantumCircuit):
    self.convert_step_1(quantumCircuit)

    n_wires = self.n_inputs + self.n_outputs + self.n_internal

    for i, output_gate in enumerate(self.output_gates):
        a = output_gate
        a_prime = n_wires + i
        quantumCircuit.cx(a, a_prime)

    quantumCircuit.barrier()
    self.convert_step_2(quantumCircuit)

ClassicalCircuit.convert = convert

In [5]:
cc = ClassicalCircuit("circuit.txt")
cc.print()

n_inputs: 3
n_outputs: 2
n_internal: 2
Input Gates: [0, 1, 2]
Output Gates: [3, 4]
Internal Gates: [5, 6]
Gates: 
[5, 'and', 0, 1]
[3, 'not', 5]
[6, 'not', 2]
[4, 'and', 5, 6]



In [6]:

n_wires = cc.n_inputs + cc.n_outputs + cc.n_internal
qc = QuantumCircuit(n_wires, 0)
cc.convert_step_1(qc)
print(qc)
print()

           ░            ░            ░       ░ 
q_0: ──■───░────────────░────────────░───────░─
       │   ░            ░            ░       ░ 
q_1: ──■───░────────────░────────────░───────░─
       │   ░            ░            ░       ░ 
q_2: ──┼───░────────────░────────■───░───────░─
       │   ░ ┌───┐┌───┐ ░        │   ░       ░ 
q_3: ──┼───░─┤ X ├┤ X ├─░────────┼───░───────░─
       │   ░ └───┘└─┬─┘ ░        │   ░ ┌───┐ ░ 
q_4: ──┼───░────────┼───░────────┼───░─┤ X ├─░─
     ┌─┴─┐ ░        │   ░        │   ░ └─┬─┘ ░ 
q_5: ┤ X ├─░────────■───░────────┼───░───■───░─
     └───┘ ░            ░ ┌───┐┌─┴─┐ ░   │   ░ 
q_6: ──────░────────────░─┤ X ├┤ X ├─░───■───░─
           ░            ░ └───┘└───┘ ░       ░ 



In [7]:
n_wires = cc.n_inputs + cc.n_outputs + cc.n_internal
qc = QuantumCircuit(n_wires, 0)
cc.convert_step_2(qc)
print(qc)
print()

           ░            ░            ░       ░ 
q_0: ──────░────────────░────────────░───■───░─
           ░            ░            ░   │   ░ 
q_1: ──────░────────────░────────────░───■───░─
           ░            ░            ░   │   ░ 
q_2: ──────░───■────────░────────────░───┼───░─
           ░   │        ░ ┌───┐┌───┐ ░   │   ░ 
q_3: ──────░───┼────────░─┤ X ├┤ X ├─░───┼───░─
     ┌───┐ ░   │        ░ └─┬─┘└───┘ ░   │   ░ 
q_4: ┤ X ├─░───┼────────░───┼────────░───┼───░─
     └─┬─┘ ░   │        ░   │        ░ ┌─┴─┐ ░ 
q_5: ──■───░───┼────────░───■────────░─┤ X ├─░─
       │   ░ ┌─┴─┐┌───┐ ░            ░ └───┘ ░ 
q_6: ──■───░─┤ X ├┤ X ├─░────────────░───────░─
           ░ └───┘└───┘ ░            ░       ░ 



In [8]:
n_wires = cc.n_inputs + 2 * cc.n_outputs + cc.n_internal
qc = QuantumCircuit(n_wires, 0)
cc.convert(qc)
print(qc)
print()

           ░            ░            ░       ░            ░       ░           »
q_0: ──■───░────────────░────────────░───────░────────────░───────░───────────»
       │   ░            ░            ░       ░            ░       ░           »
q_1: ──■───░────────────░────────────░───────░────────────░───────░───────────»
       │   ░            ░            ░       ░            ░       ░           »
q_2: ──┼───░────────────░────────■───░───────░────────────░───────░───■───────»
       │   ░ ┌───┐┌───┐ ░        │   ░       ░            ░       ░   │       »
q_3: ──┼───░─┤ X ├┤ X ├─░────────┼───░───────░───■────────░───────░───┼───────»
       │   ░ └───┘└─┬─┘ ░        │   ░ ┌───┐ ░   │        ░ ┌───┐ ░   │       »
q_4: ──┼───░────────┼───░────────┼───░─┤ X ├─░───┼────■───░─┤ X ├─░───┼───────»
     ┌─┴─┐ ░        │   ░        │   ░ └─┬─┘ ░   │    │   ░ └─┬─┘ ░   │       »
q_5: ┤ X ├─░────────■───░────────┼───░───■───░───┼────┼───░───■───░───┼───────»
     └───┘ ░            ░ ┌───┐┌─┴─┐ ░  

## 2.6 Controlled Uf

In [9]:
def convert_contr_step_1(self, quantumCircuit, i):
    for gate in self.gates:
        match gate[1]:
            case "and":
                # controlled-CCX (controlled-Toffoli) = 3-controlled X
                a, b, c = gate[0], gate[2], gate[3]
                quantumCircuit.mcx([i, b, c], a)
            case "not":
                # controlled-X and controlled-CX
                a, b = gate[0], gate[2]
                quantumCircuit.cx(i, a)
                quantumCircuit.ccx(i, b, a)
            case "or":
                # bonus, controlled version of or gate
                a, b, c = gate[0], gate[2], gate[3]
                quantumCircuit.cx(i, b)
                quantumCircuit.cx(i, c)
                quantumCircuit.mcx([i, b, c], a)
                quantumCircuit.cx(i, a)
                quantumCircuit.cx(i, b)
                quantumCircuit.cx(i, c)
            case "xor":
                # bonus, controlled version of xor gate
                a, b, c = gate[0], gate[2], gate[3]
                quantumCircuit.ccx(i, b, a)
                quantumCircuit.ccx(i, c, a)
            case "nand":
                # bonus, controlled version of nand gate
                a, b, c = gate[0], gate[2], gate[3]
                quantumCircuit.mcx([i, b, c], a)
                quantumCircuit.cx(i, a)
        quantumCircuit.barrier()

ClassicalCircuit.convert_contr_step_1 = convert_contr_step_1

In [10]:
def convert_contr_step_2(self, quantumCircuit, i):
    for gate in reversed(self.gates):
        match gate[1]:
            case "and":
                # controlled-CCX (controlled-Toffoli) = 3-controlled X
                a, b, c = gate[0], gate[2], gate[3]
                quantumCircuit.mcx([i, b, c], a)
            case "not":
                # controlled-CX and controlled-X (reversed)
                a, b = gate[0], gate[2]
                quantumCircuit.ccx(i, b, a)
                quantumCircuit.cx(i, a)
            case "or":
                # bonus, controlled version of or gate (reversed)
                a, b, c = gate[0], gate[2], gate[3]
                quantumCircuit.cx(i, c)
                quantumCircuit.cx(i, b)
                quantumCircuit.cx(i, a)
                quantumCircuit.mcx([i, b, c], a)
                quantumCircuit.cx(i, c)
                quantumCircuit.cx(i, b)
            case "xor":
                # bonus, controlled version of xor gate (reversed)
                a, b, c = gate[0], gate[2], gate[3]
                quantumCircuit.ccx(i, c, a)
                quantumCircuit.ccx(i, b, a)
            case "nand":
                # bonus, controlled version of nand gate (reversed)
                a, b, c = gate[0], gate[2], gate[3]
                quantumCircuit.cx(i, a)
                quantumCircuit.mcx([i, b, c], a)
        quantumCircuit.barrier()

ClassicalCircuit.convert_contr_step_2 = convert_contr_step_2

In [11]:
def convert_contr(self, quantumCircuit, i):
    self.convert_contr_step_1(quantumCircuit, i)

    n_wires = self.n_inputs + self.n_outputs + self.n_internal

    for j, output_gate in enumerate(self.output_gates):
        a = output_gate
        a_prime = n_wires + j
        quantumCircuit.ccx(i, a, a_prime)

    quantumCircuit.barrier()
    
    self.convert_contr_step_2(quantumCircuit, i)

ClassicalCircuit.convert_contr = convert_contr

In [12]:
n_wires = cc.n_inputs + 2 * cc.n_outputs + cc.n_internal + 1  # added one for the control qubit
qc_controlled = QuantumCircuit(n_wires, 0)

control_qubit = n_wires - 1

qc_controlled = QuantumCircuit(n_wires, 0)
cc.convert_contr(qc_controlled, control_qubit)
print(qc_controlled)
print()

           ░            ░            ░       ░            ░       ░           »
q_0: ──■───░────────────░────────────░───────░────────────░───────░───────────»
       │   ░            ░            ░       ░            ░       ░           »
q_1: ──■───░────────────░────────────░───────░────────────░───────░───────────»
       │   ░            ░            ░       ░            ░       ░           »
q_2: ──┼───░────────────░────────■───░───────░────────────░───────░───■───────»
       │   ░ ┌───┐┌───┐ ░        │   ░       ░            ░       ░   │       »
q_3: ──┼───░─┤ X ├┤ X ├─░────────┼───░───────░───■────────░───────░───┼───────»
       │   ░ └─┬─┘└─┬─┘ ░        │   ░ ┌───┐ ░   │        ░ ┌───┐ ░   │       »
q_4: ──┼───░───┼────┼───░────────┼───░─┤ X ├─░───┼────■───░─┤ X ├─░───┼───────»
     ┌─┴─┐ ░   │    │   ░        │   ░ └─┬─┘ ░   │    │   ░ └─┬─┘ ░   │       »
q_5: ┤ X ├─░───┼────■───░────────┼───░───■───░───┼────┼───░───■───░───┼───────»
     └─┬─┘ ░   │    │   ░ ┌───┐┌─┴─┐ ░  